In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

# random seed for always same results
tf.random.set_seed(678)

import numpy as np

# Inputs and Labels

In [5]:
X = np.array([[0., 0.], [0., 1.], [1., 0.], [1., 1.]])
Y = np.array([0., 1., 1., 0.])

# Two dense Layers

In [6]:
model = Sequential()
# first dense layer
model.add(Dense(units = 2, activation = 'sigmoid', input_dim = 2))
# second dense layer
model.add(Dense(units = 1, activation = 'sigmoid'))

c:\주 폴더\학교\2024\세특\학술제\제출 파일\ClimateAI\AcademicFestival\ssgVenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 2)              │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9 (36.00 B)

 Trainable params: 9 (36.00 B)

 Non-trainable params: 0 (0.00 B)

In [8]:
# loss function and optimization
model.compile(loss = 'binary_crossentropy', optimizer = 'sgd', metrics = ['accuracy'])

# Train

In [ ]:
'''
주의!!!!!
원래 이 코드의 epochs = 50000이고 이것을 그냥
일반 컴퓨터 cpu로 하면 학습 시간이 너무 깁니다!
코랩의 좋은 GPU가 있을 때만 50000번으로 시도하세요!
'''
# Train
# model.fit(X, Y, epochs = 1000, batch_size = 4, verbose = 0)

In [21]:
# model.save("Dense_tf.h5")

# Test

[0, 0], [0, 1] outputs are less than 0.5,
[0, 1], [1, 0] outputs are greater than 0.5

In [9]:
from tensorflow.keras.models import load_model
# colab에서 더 많은 epochs(10000)로 학습한 녀석
model = load_model("Dense_tf.keras")

print(model.predict(X, batch_size = 4))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
[[0.04725352]
 [0.9301207 ]
 [0.9293673 ]
 [0.12402218]]


# printing First Dense Layer's weights and bias

In [10]:
print("first layer weights: ",model.layers[0].get_weights()[0])
print("first layer bias: ",model.layers[0].get_weights()[1])

first layer weights:  [[-3.4563856 -5.6441655]
 [-3.4797676 -5.8310213]]
first layer bias:  [5.021999  2.0357053]


여기서 주의할 점은, -3.4563856과 -3.4797676이 첫 번째 노드의 가중치이다. 한 리스트로 묶인 것이 첫 번째 노드 가중치가 아니다. 이것 때문에 이후 수행하는 직접 만들기에서는 전치 행렬을 활용하는 것이다.

# Print Second Dense layer's weights and bias

In [24]:
print("second layer weights: ",model.layers[1].get_weights()[0])
print("second layer bias: ",model.layers[1].get_weights()[1])

second layer weights:  [[ 6.7809935]
 [-7.8176737]]
second layer bias:  [-2.8257167]


# calculate yourself and verify same result with TF dense layers

tensorflow 제공 기능을 손으로 직접 구현해 보아요~~~

이걸 해보면 알겠지만, tensorflow에서 제공하는 기능이 얼마나 쌈@뽕한지 알 수 있음

In [1]:
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [19]:
print(model.layers[0].get_weights())

[array([[-3.4563856, -5.6441655],
       [-3.4797676, -5.8310213]], dtype=float32), array([5.021999 , 2.0357053], dtype=float32)]


In [20]:
def get_output(x):
    # first dense layer, first node output
    layer0 = model.layers[0]
    layer0_weights, layer0_bias = layer0.get_weights()
    # 기존의 첫 번째 노드 가중치가 흩어져 있는 관계로 행렬을 전치시킨 후 값을 가져온다.
    layer0_node0_weights = np.transpose(layer0_weights)[0]
    layer0_node0_bias = layer0_bias[0]
    # np.dot()을 통해 1차원 행렬인 벡터 간의 곱, 즉 벡터 내적을 수행함. -> 시그모이드까지 거쳐서 하나의 상수가 나옴.
    layer0_node0_output = sigmoid(np.dot(x, layer0_node0_weights) + layer0_node0_bias)

    # first dense layer, second node output
    layer0_node1_weights = np.transpose(layer0_weights)[1]
    layer0_node1_bias = layer0_bias[1]
    layer0_node1_output = sigmoid(np.dot(x, layer0_node1_weights) + layer0_node1_bias)

    # second layer output
    layer1 = model.layers[1]
    layer1_weights, layer1_bias = layer1.get_weights()
    layer1_output = sigmoid(np.dot([layer0_node0_output, layer0_node1_output], layer1_weights) + layer1_bias)

    print(layer1_output)

In [22]:
get_output([0, 0])

0.04725352759072789


C:\Users\Home\AppData\Local\Temp\ipykernel_11192\1501651968.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1 / (1 + math.exp(-x))


In [23]:
get_output([0,1])

0.9301206434294983


C:\Users\Home\AppData\Local\Temp\ipykernel_11192\1501651968.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1 / (1 + math.exp(-x))


In [24]:
get_output([1,0])

0.9293672902232236


C:\Users\Home\AppData\Local\Temp\ipykernel_11192\1501651968.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1 / (1 + math.exp(-x))


In [25]:
get_output([1,1])

0.1240222239085602


C:\Users\Home\AppData\Local\Temp\ipykernel_11192\1501651968.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1 / (1 + math.exp(-x))
